## General Import and Setup

In [20]:
import pandas as pd
import plotly.express as px
from collections import Counter
import string
import spacy
from textblob_de import TextBlobDE
from stopwordsiso import stopwords
import plotly.graph_objects as go
import plotly
import de_core_news_sm

Bedingt durch die Unzulänglichkeiten der importierten StopWordsISO Liste definieren wir hier weitere Stopwörter, welche im Trial and Error Verfahren festgestellt wurden.

In [10]:
custom_stops = ["'ne", "ne", "komm", "...", "lass", "yeah", "sag", "einfach", "weiß", "ah", "ey", "all", "mach", "bleibt", "heut", "nem", "seh", "sehn", "sehen", "geh", "gehn", "gehen", "raus", "egal", "wär", "sagen", "halt", "guck", "un"]

In [11]:
nlp = spacy.load("de_core_news_sm")
data = pd.read_csv('../data/raw/songs_complete_final.csv')
data.head()

,artist,artist_id,album,album_id,release_date,title,full_title,song_id,lyrics,release_year,weekday,genre,genre_cat,word_count
0,1099,209826,10999,831505,2021-10-29,INTRO (10999),INTRO (10999) by King Khalil,7337100,"Powpow Dicka, das kein Rap mehr, das ist Kind...",2021,Friday,post-rock,Rock,60
1,1099,209826,10999,831505,2021-10-29,GIB IHM,GIB IHM by King Khalil & AK 33,7337102,"Gefährliche, gefährliche KiKiKiKi Gefährliche ...",2021,Friday,post-rock,Rock,92
2,1099,209826,KING KONG,546439,2020-01-31,BUNDESWEIT,BUNDESWEIT by King Khalil (Ft. Fler),4532899,Aus meiner Stadt fliegen Leuchtclips und Bresl...,2020,Friday,post-rock,Rock,113
3,1099,209826,KING KONG,546439,2020-01-31,HOLLANDA,HOLLANDA by King Khalil (Ft. Mert),5189158,Because youre so sweet You lift up my heart An...,2020,Friday,post-rock,Rock,80
4,1099,209826,KING KONG,546439,2020-01-31,MOON,MOON by King Khalil & Lil Lano,5109178,. Liquid Swords GZA Actual . souljaboytellem...,2020,Friday,post-rock,Rock,233


### Checking the Vocabulary of the Artists

Die Darstellung zeigt einen Scatterplot der Songs und der insgesamt verwendeten Worte eines Künstlers. Eine Trendlinie basierend auf den Gesamtdaten ist eingezeichnet. Tendenziell haben Künstler*innen links über der Trendlinie einen kleineren Wortschatz als solche, welche unterhalb der Linie liegen.

In [21]:
grouped_data = data.groupby("artist")

tokenized_words = grouped_data["lyrics"].apply(lambda x: ' '.join(x).split())

unique_word_counts = tokenized_words.apply(lambda x: len(x))

num_songs = grouped_data.size()


df = pd.DataFrame({'Interpret': unique_word_counts.index, 'Word Count': unique_word_counts, 'Number of Songs': num_songs})

fig = px.scatter(df, x='Word Count', y='Number of Songs', hover_data=['Interpret'], title='Number of Songs vs. Word Count', trendline="ols")

fig.show()

### Checking the 10 most used words for each Genre

In [8]:
def filter_stopwords(tokens):
    german_stopwords = set(stopwords(["de"]))
    full_Stopwords = list(german_stopwords) + custom_stops
    #Zusammenführen der Stopword-Listen
    filtered_tokens = []
    for token in tokens:
        if token.lower_ not in full_Stopwords and token.text not in string.punctuation and not token.is_space:
            filtered_tokens.append(token.lower_)
    return filtered_tokens

In [9]:
data["filtered_lyrics"] = data["lyrics"].apply(lambda x: filter_stopwords(nlp(x)))

In [10]:
data["word_counts"] = data["filtered_lyrics"].apply(lambda x: Counter(set(x)))

In [11]:
grouped_data = data.groupby("genre_cat")

In [12]:
genre_word_counts = {}
for genre_cat, group in grouped_data:
    word_counts = group["word_counts"].sum()
    genre_word_counts[genre_cat] = word_counts

top_10_words_per_genre = {}
for genre, word_counts in genre_word_counts.items():
    # Exclude whitespace from the top words list
    word_counts.pop('', None)
    # Sort the words based on song count only
    top_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:10]
    top_10_words_per_genre[genre] = top_words

for genre, top_words in top_10_words_per_genre.items():
    print("Genre:", genre)
    df = pd.DataFrame(top_words, columns=["Word", "Song Count"])
    print(df)
    print()

Genre: Pop
    Word  Song Count
0  leben         700
1   welt         582
2  liebe         506
3  nacht         485
4   herz         436
5  augen         352
6   kopf         320
7   klar         261
8  licht         257
9   hand         254

Genre: Rap
     Word  Song Count
0   leben        2524
1    welt        1594
2    geld        1568
3    fick        1444
4    kopf        1417
5  scheiß        1367
6   nacht        1270
7    paar        1238
8  rapper        1204
9   bitch        1141

Genre: Rock
    Word  Song Count
0  leben         588
1   welt         549
2  nacht         349
3   herz         306
4  liebe         293
5  augen         259
6  schön         222
7   hand         221
8  stadt         197
9  licht         195

Genre: Schlager
     Word  Song Count
0   nacht         441
1   liebe         439
2   leben         386
3    herz         369
4    welt         359
5  himmel         197
6   glück         192
7   augen         192
8   schön         150
9  vorbei         143



Während Pop, Schlager und Rock sich ziemlich ähnlich sehen, sticht Rap deutlich hervor. Hier finden sich die größten Abweichungen (s. Abb.: newplot.png).

In [17]:
#Plotly benötigt ein Dataframe anstelle des erstellten Dictionairies
df = pd.DataFrame(top_10_words_per_genre)

table = go.Table(
    header=dict(values=df.columns),
    cells=dict(values=df.transpose().values.tolist())
)

layout = go.Layout(
    title='Top 10 Words per Genre',
    height=450
)

fig = go.Figure(data=[table], layout=layout)

fig.show()

### Sentiment Analysis of a Genre

Mit TextBlobDE führen wir hier eine Sentiment Analysis der Liedtexte durch. Als Beispiel sind die ersten 5 Texte gegeben. Man sieht noch keine sehr deutlichen Ausschläge.

In [44]:
genre_data = data[data["genre"] == "Rap"]

sentiments = []
for lyric in genre_data["lyrics"]:
    blob = TextBlobDE(lyric)
    sentiment = blob.sentiment
    sentiments.append((lyric, sentiment.polarity, sentiment.subjectivity))

sentiments_df = pd.DataFrame(sentiments, columns=["Lyric", "Polarity", "Subjectivity"])
print(sentiments_df.head())

                                               Lyric  Polarity  Subjectivity
0   Okay, Ansage, Freundchen, ich warne dich Ey, ...  0.067500      0.100000
1    Ey Leute, passt doch auf, ihr lebt in Gefahr...  0.115972      0.006944
2   Okay, Ellen lang und elegant Keule Mutterfick... -0.031250      0.015625
3   Hallihallöchen, ist lange her Schatz Ich vers...  0.048352      0.000000
4    Ich steh auf am Kleister riechen und gleiche... -0.054167      0.144444


In [19]:
def analyze_sentiment(data, genre):
    genre_data = data[data["genre_cat"] == genre]

    sentiments = []

    for lyric in genre_data["lyrics"]:
        blob = TextBlobDE(lyric)
        sentiment = blob.sentiment.polarity
        sentiments.append(sentiment)

    return sentiments

grouped_data = data.groupby("genre_cat")
all_sentiments = []

for genre, group in grouped_data:
    sentiments = analyze_sentiment(data, genre)
    all_sentiments.append(sentiments)

fig = go.Figure()

for genre, sentiments in zip(data["genre_cat"].unique(), all_sentiments):
    fig.add_trace(go.Box(y=sentiments, name=genre))

fig.update_layout(
    title="Sentiment Analysis by Genre",
    xaxis=dict(title="Genre"),
    yaxis=dict(title="Sentiment Polarity")
)

fig.show()


Daher Plotten wir das ganze als Boxplots, welche einen deutlich leichteren Überblick über die Gesamtdaten geben. Überrachend ist, dass Pop deutlich Negativer abschneidet als Rap oder Rock. Schlager hingegen scheint tatächlich deutlich Positiver zu sein als die anderen Genres. 

Jedoch ist hier noch klar zu erwähnen das es sich lediglich um eine Rudimentäre Analyse handelt, da die Capabilities mit Huggingface und den eigens trainierten Modellen noch nicht zum Einsatz gebracht werden konnten.